In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install -U sacremoses
# !pip install ftfy
# !pip install imblearn

In [2]:
import numpy as np
import pandas as pd

# from sacremoses import MosesTokenizer
from sklearn.pipeline import FeatureUnion#, Pipeline
from imblearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.naive_bayes import GaussianNB, ComplementNB
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import precision_score, recall_score, f1_score
from scipy.stats import uniform, loguniform
from imblearn.over_sampling import RandomOverSampler,SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

In [3]:
df = pd.read_csv('Data\mbti_agg_clean_1000.csv', usecols=['post', 'type', 'type_self_reported']).rename({'post': 'posts'},axis=1)
df.head()

,posts,type,type_self_reported
0,'<|url_youtube|><|sep|><|url_tumblr|><|sep|>en...,INFJ,NaN
1,'I'm finding the lack of me in these posts ver...,ENTP,NaN
2,'Good one _____ <|url_youtube|><|sep|>Of co...,INTP,NaN
3,"'Dear INTP, I enjoyed our conversation the o...",INTJ,NaN
4,'You're fired.<|sep|>That's another silly misc...,ENTJ,NaN


In [7]:
df = df.rename({'posts': 'X1'}, axis=1)
df['X2'] = (df['type_self_reported'].str.get(0)=='E').astype(int)
df['Y'] = (df['type'].str.get(0)=='E').astype(int)
df = df[['X1', 'X2', 'Y']]

df.head()

,X1,X2,Y
0,'<|url_youtube|>,0,0
1,<|url_tumblr|>,0,0
2,enfp and intj moments <|url_youtube|> sports...,0,0
3,What has been the most life-changing experienc...,0,0
4,<|url_youtube|> <|url_youtube|> On repeat f...,0,0


In [8]:
df_train, df_test = train_test_split(df, test_size=0.3, stratify=df['Y'], random_state=42)
df_train

,X1,X2,Y
288138,"well as bad as it is, animal testing is the on...",0,0
193575,Hey guys! I have been friends with an ESFJ ma...,0,1
34725,Why? Because our Te supports all the conclusio...,0,0
13381,I don't even know at this point. I've always f...,0,0
62088,So when I was younger (high school) I would al...,0,1
...,...,...,...
387346,"Ahhhh, that's interesting..what's it like bein...",0,1
231438,Wait...SANTA'S NOT REAL??????!!!!!!!!!!!!!!!!!...,0,0
418820,As an INFP I use my Tertiary Si like a lens th...,0,0
173453,I really like music and listen to it on a regu...,0,0


In [ ]:
df_test

,X1,X2,Y
2611,'the maltese falcon........ and the usual susp...,0,0
2620,"'Oh yes, it's so fantastically true it's wonde...",0,1
8408,'I was wondering as an INFP how do you react t...,0,0
1097,'1. Be Realistic: This is not to say that you ...,0,0
8083,"'Well, I should be writing a conclusion to thi...",0,0
...,...,...,...
6253,'The giggle fits. Oh my god the giggle fits. ...,0,0
6268,'Thinking about transgender stuff... It's har...,0,0
593,'so im an istp male - though my profile says e...,0,1
692,"'I can't remember exactly, but it was an infor...",0,0


In [ ]:
# mt = MosesTokenizer()
# moses = lambda x: mt.tokenize(x, protected_patterns=[r'<\|[^>]+\|>'])

In [ ]:
# pipeline = Pipeline([
#     ('tfidf', TfidfVectorizer(ngram_range=(1,2), max_df=0.5, min_df=0.1, token_pattern=r'<\|[^>]+\|>|(?u)\b\w\w+\b')),
#     ('resample', SMOTE(random_state=42)),
#     ('clf', LogisticRegression())
# ])

pipeline = Pipeline([
    
    ('tfidf', TfidfVectorizer(ngram_range=(1,2), max_df=0.5, min_df=0.1, token_pattern=r'<\|[^>]+\|>|(?u)\b\w\w+\b')),
    ('resample', SMOTE(random_state=42)),
    ('clf', LogisticRegression())
])
pipeline.fit(df_train['X'], df_train['Y'])

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.5, min_df=0.1, ngram_range=(1, 2),
                                 token_pattern='<\\|[^>]+\\|>|(?u)\\b\\w\\w+\\b')),
                ('resample', SMOTE(random_state=42)),
                ('clf', LogisticRegression())])

In [ ]:
pipeline.score(df_test['X'], df_test['Y'])

0.841336918939685

In [ ]:
f1_score(df_test['Y'], pipeline.predict(df_test['X']))

0.6709163346613546

In [ ]:
pipeline = Pipeline([
    ('vec', CountVectorizer(ngram_range=(1,2), max_df=0.5, min_df=0.1, token_pattern=r'<\|[^>]+\|>|(?u)\b\w\w+\b')),
    ('resample', RandomOverSampler(random_state=42)),
    ('clf', ComplementNB())
])
search = RandomizedSearchCV(pipeline, param_distributions={
    'vec__max_df': uniform(loc=0.25, scale=0.75),
    'vec__min_df': uniform(loc=0, scale=0.25),
}, cv=5, scoring='f1', n_iter=5, verbose=10, n_jobs=4)

search.fit(df_train['X'], df_train['Y'])

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:   28.2s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   42.4s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:  1.2min
[Parallel(n_jobs=4)]: Done  21 out of  25 | elapsed:  1.5min remaining:   17.0s
[Parallel(n_jobs=4)]: Done  25 out of  25 | elapsed:  1.7min finished


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('vec',
                                              CountVectorizer(max_df=0.5,
                                                              min_df=0.1,
                                                              ngram_range=(1,
                                                                           2),
                                                              token_pattern='<\\|[^>]+\\|>|(?u)\\b\\w\\w+\\b')),
                                             ('resample',
                                              RandomOverSampler(random_state=42)),
                                             ('clf', ComplementNB())]),
                   n_iter=5, n_jobs=4,
                   param_distributions={'vec__max_df': <scipy.stats._distn_infrastructure.rv_frozen object at 0x2a8780a50>,
                                        'vec__min_df': <scipy.stats._distn_infrastructure.rv_frozen object at 0x2ba9f9590>},
   

In [ ]:
pd.DataFrame(search.cv_results_).sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_vec__max_df,param_vec__min_df,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
1,11.838682,0.045395,1.875424,0.033777,0.452946,0.189968,"{'vec__max_df': 0.4529463041161192, 'vec__min_...",0.603269,0.625749,0.606335,0.569733,0.607784,0.602574,0.018206,1
2,12.033462,0.134764,2.631917,0.401602,0.491486,0.00743156,"{'vec__max_df': 0.4914858561491164, 'vec__min_...",0.599251,0.603978,0.580762,0.559567,0.589065,0.586525,0.015703,2
3,12.779804,0.353503,2.372186,0.137064,0.38796,0.204178,"{'vec__max_df': 0.3879596504720168, 'vec__min_...",0.569254,0.610778,0.597201,0.571429,0.578313,0.585395,0.016059,3
4,13.344455,1.197899,2.292739,0.172988,0.298464,0.117432,"{'vec__max_df': 0.29846388707237154, 'vec__min...",0.571865,0.590214,0.582435,0.552147,0.545455,0.568423,0.017178,4
0,11.836600,0.060421,2.089289,0.135410,0.270039,0.204644,"{'vec__max_df': 0.27003915825245073, 'vec__min...",0.448980,0.470948,0.438569,0.464122,0.451807,0.454885,0.011436,5


In [ ]:
search.best_estimator_, search.best_params_

(Pipeline(steps=[('vec',
                  CountVectorizer(max_df=0.4529463041161192,
                                  min_df=0.18996834260014261, ngram_range=(1, 2),
                                  token_pattern='<\\|[^>]+\\|>|(?u)\\b\\w\\w+\\b')),
                 ('resample', RandomOverSampler(random_state=42)),
                 ('clf', ComplementNB())]),
 {'vec__max_df': 0.4529463041161192, 'vec__min_df': 0.18996834260014261})

In [ ]:
search.score(df_test['X'], df_test['Y'])

0.5968939905469278

In [ ]:
f1_score(df_test['Y'], search.predict(df_test['X']))

0.5968939905469278